In [1]:
from pipelines.preprocessing_images import extract_features_from_images_text

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2025-01-27 03:57:40.587885: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
image_features, text_features, labels, restaurants, photo_ids = extract_features_from_images_text(2, 
"sagemaker-studio-619071335465-8h7owh9eftx",
'samples/image classification/',
'samples/image classification/final_image_to_text_results_other.csv',
'image datasets/other images/',
"/home/sagemaker-user/clip_model",
"/home/sagemaker-user/clip_processor")

In [4]:
from torch import nn
import torch

class MultimodalClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate):
        super(MultimodalClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim * 2, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, image_embedding, text_embedding):
        x = torch.cat((image_embedding, text_embedding), dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cuda" if torch.cuda.is_available() else "cpu"

input_dim = 512
hidden_dim = 256
dropout_rate = 0.45584099758281393

local_multimodal_model_path = "/home/sagemaker-user/model/multimodal_classifier_model.pth"


multimodal_classifier = MultimodalClassifier(input_dim, hidden_dim, dropout_rate).to(device)
multimodal_classifier.load_state_dict(torch.load(local_multimodal_model_path, map_location=device))
multimodal_classifier.eval()

state_dict = torch.load(local_multimodal_model_path, map_location=device)

# Load the state dictionary into the model
multimodal_classifier.load_state_dict(state_dict)

# Print state_dict
print(state_dict)

OrderedDict([('fc1.weight', tensor([[ 4.1057e-03,  8.4955e-04, -3.3628e-03,  ..., -2.6028e-03,
          7.6136e-04, -2.7815e-04],
        [-2.7187e-03, -1.8087e-03,  2.2379e-03,  ...,  4.7140e-03,
          1.0096e-03,  1.6289e-04],
        [ 3.5197e-03,  5.2734e-03, -6.2515e-03,  ..., -2.2891e-03,
         -1.6100e-03, -2.6531e-03],
        ...,
        [ 5.7256e-03,  3.6784e-03, -5.2710e-03,  ..., -3.3767e-03,
         -1.5594e-04, -2.4984e-03],
        [-3.9317e-03, -5.1555e-05,  4.2796e-03,  ...,  2.0974e-03,
         -7.9344e-04,  6.9403e-04],
        [ 9.2486e-44, -8.5479e-44, -7.9874e-44,  ..., -4.6243e-44,
          8.8282e-44,  9.6690e-44]])), ('fc1.bias', tensor([ 6.6227e-03, -5.1327e-04,  6.2926e-03,  2.0024e-20,  4.5280e-03,
         5.4196e-03,  6.8448e-03, -2.5870e-17,  6.0964e-03,  3.3327e-03,
         5.8031e-03, -1.2752e-03,  4.5492e-03, -1.6060e-03,  1.1419e-05,
         5.7791e-03,  3.4429e-03,  4.7352e-03, -1.9316e-20, -3.1827e-04,
         5.1584e-03, -1.2274e-03,

/tmp/ipykernel_12857/161036892.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  multimodal_classifier.load_state_dict(torch.load(local_multimodal_model_path, map_locatio

In [5]:
def predict_multiple(image_embeddings, text_embeddings, labels, restaurants, photo_ids):
    predictions = []
    true_labels = []
    results = []  # Store the combined output for each sample

    for i in range(image_embeddings.size(0)):
        image_embedding = image_embeddings[i].unsqueeze(0)
        text_embedding = text_embeddings[i].unsqueeze(0)

        with torch.no_grad():
            prediction = multimodal_classifier(image_embedding, text_embedding)
            prediction_score = prediction.squeeze().item()  
            predicted_label = "Positive" if prediction_score > 0.5 else "Negative"

        true_label = "Positive" if labels[i].item() == 1 else "Negative"
        predictions.append((predicted_label, prediction_score))
        true_labels.append(true_label)

        # Append the current sample's results
        results.append({
            "restaurant_name": restaurants[i],
            "photo_id": photo_ids[i],
            "true_label": true_label,
            "predicted_label": predicted_label,
            "prediction_score": prediction_score
        })

    return results


results = predict_multiple(image_features, text_features, labels, restaurants, photo_ids)

# Print the results
for i, result in enumerate(results):
    print(f"Sample {i + 1}:")
    print(f"Restaurant Name: {result['restaurant_name']}")
    print(f"Photo ID: {result['photo_id']}")
    print(f"True Label: {result['true_label']}")
    print(f"Prediction: {result['predicted_label']} (Score: {result['prediction_score']:.4f})")
    print("-" * 40)

Sample 1:
Restaurant Name: Broussard's
Photo ID: 2CmoBjiw0GYOCbakHT3dHA.jpg
True Label: Positive
Prediction: Positive (Score: 0.5449)
----------------------------------------
Sample 2:
Restaurant Name: Biscuit Love: Gulch
Photo ID: 5zX5vPMu6gYiPbxvM7jXrQ.jpg
True Label: Negative
Prediction: Negative (Score: 0.0326)
----------------------------------------


In [9]:
import tarfile
import os

# Paths to your files
model_file_path = "/home/sagemaker-user/model/multimodal_classifier_model.pth"
inference_file_path = "/home/sagemaker-user/model/inference.py"

# Path to save the tar.gz file
tar_file_path = "/home/sagemaker-user/model/multimodal_classifier_model.tar.gz"

# Create a tar.gz file
with tarfile.open(tar_file_path, "w:gz") as tar:
    # Add the model file to the tar.gz archive
    model_name = os.path.basename(model_file_path)  # Extract file name from path
    tar.add(model_file_path, arcname=model_name)

    # Add the inference.py file to the tar.gz archive
    inference_name = os.path.basename(inference_file_path)  # Extract file name from path
    tar.add(inference_file_path, arcname=inference_name)

print(f"tar.gz file created at: {tar_file_path}")


tar.gz file created at: /home/sagemaker-user/model/multimodal_classifier_model.tar.gz


In [10]:
import boto3

s3 = boto3.client('s3')
s3_bucket = "sagemaker-studio-619071335465-8h7owh9eftx"
s3_key = "opt/ml/model/multimodal_classifier_model.tar.gz"

# Upload the tar.gz file to S3
s3.upload_file(tar_file_path, s3_bucket, s3_key)

print(f"Model uploaded to s3://{s3_bucket}/{s3_key}")


Model uploaded to s3://sagemaker-studio-619071335465-8h7owh9eftx/opt/ml/model/multimodal_classifier_model.tar.gz


In [11]:
from sagemaker import get_execution_role, Session
from sagemaker.pytorch import PyTorchModel

sagemaker_session = Session()
role = get_execution_role()

model_artifact = f"s3://{s3_bucket}/{s3_key}"

print(model_artifact)
print(torch.__version__)

s3://sagemaker-studio-619071335465-8h7owh9eftx/opt/ml/model/multimodal_classifier_model.tar.gz
2.4.1.post100


In [12]:

from sagemaker.predictor import Predictor

pytorch_model = PyTorchModel( 
    model_data=model_artifact,
    role=role,
    sagemaker_session=sagemaker_session,
    framework_version="2.1.0",    # Match your PyTorch version
    py_version="py310"
)

# Deploy the model to an endpoint
endpoint_name = "multimodal-classifier-endpoint-0126257"
predictor = pytorch_model.deploy(
    instance_type="ml.m5.large",
    initial_instance_count=1,
    endpoint_name=endpoint_name
)

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session
)

print(f"Model deployed to endpoint: {predictor.endpoint_name}")


------!Model deployed to endpoint: multimodal-classifier-endpoint-0126257


In [20]:
import boto3

sagemaker_client = boto3.client("sagemaker")
endpoint_name = endpoint_name

response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {response['EndpointStatus']}")

Endpoint Status: InService


In [17]:
import json
import base64
from sagemaker.predictor import Predictor

print(f"image_features type: {type(image_features)}, dtype: {image_features.dtype}")
print(f"text_features type: {type(text_features)}, dtype: {text_features.dtype}")


input_data = {
    "image_embedding": image_features.tolist(),
    "text_embedding": text_features.tolist()
}

response = predictor.predict(
    data=json.dumps(input_data),  # Convert your input to JSON format
    initial_args={"ContentType": "application/json"}  # Set Content-Type explicitly
)

# Print the response
print(response)

image_features type: <class 'torch.Tensor'>, dtype: torch.float32
text_features type: <class 'torch.Tensor'>, dtype: torch.float32
b'[[0.5449047684669495], [0.03256279230117798]]'


In [19]:
import boto3
import json

# Initialize the SageMaker runtime client
sagemaker_client = boto3.client("sagemaker-runtime")

# Define your endpoint name
ENDPOINT_NAME = endpoint_name

# Assuming you already have image_features and text_features
print(f"image_features type: {type(image_features)}, dtype: {image_features.dtype}")
print(f"text_features type: {type(text_features)}, dtype: {text_features.dtype}")

# Prepare the payload
input_data = {
    "image_embedding": image_features.tolist(),
    "text_embedding": text_features.tolist()
}

# Call the SageMaker endpoint
response = sagemaker_client.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps(input_data)  # Convert the input to JSON format
)

# Decode and print the response
response_payload = response["Body"].read().decode("utf-8")
print("Response from SageMaker endpoint:", response_payload)



image_features type: <class 'torch.Tensor'>, dtype: torch.float32
text_features type: <class 'torch.Tensor'>, dtype: torch.float32
Response from SageMaker endpoint: [[0.5449047684669495], [0.03256279230117798]]


In [21]:
def model_fn(input_data):
    # Convert JSON to PyTorch tensors
    inputs = json.loads(input_data)
    image_embedding = torch.tensor(inputs['image_embedding'], dtype=torch.float32).to(device)
    text_embedding = torch.tensor(inputs['text_embedding'], dtype=torch.float32).to(device)

    # Verify shapes
    print(f"Image embedding shape: {image_embedding.shape}")
    print(f"Text embedding shape: {text_embedding.shape}")

    
    # Run inference
    with torch.no_grad():
        # Pass both image_embedding and text_embedding
        output = multimodal_classifier(image_embedding, text_embedding)  # Correct pass here
        
    return output.squeeze(0).tolist()

In [22]:

import json

# Initialize the SageMaker runtime client
sagemaker_runtime = boto3.client("sagemaker-runtime")

# Specify your endpoint name
endpoint_name = ENDPOINT_NAME

# Prepare a sample payload
# Replace this with actual input your model expects
sample_input = {
    "image_embedding": image_features.tolist(),  # Example embedding
    "text_embedding": text_features.tolist()   # Example embedding
}

# Serialize the payload to JSON
input_json = json.dumps(sample_input)

# print(input_json)

output = model_fn(input_json)
print("Output:", output)


Image embedding shape: torch.Size([2, 512])
Text embedding shape: torch.Size([2, 512])
Output: [[0.5449047684669495], [0.03256279230117798]]
